# **NLP FINAL PROJECT: Recipes Chatbot**

## Install, import and download of needed tools

In [1]:
!pip install -U sentence-transformers
!pip install -q xlrd
!pip install -U DialogTag

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 56.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 23.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 94.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.9 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125926 sha256=eb97ce90c16dfacda99336d15ae3cff9b3c4617b32682c11faee4f53d6346d32
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence-transformers


In [2]:
import pandas as pd
import unicodedata
import re
import random
import itertools
import nltk
import numpy as np

from sentence_transformers import SentenceTransformer, util
from itertools import chain
from nltk.stem import WordNetLemmatizer
from dialog_tag import DialogTag

nltk.download('wordnet')
nltk.download('omw-1.4')
lemmatizer = WordNetLemmatizer()
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
dialog_model = DialogTag('distilbert-base-uncased')


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


Model not found in cache. Downloading...


100%|██████████| 268M/268M [00:08<00:00, 32.7MB/s]


Some layers from the model checkpoint at /root/.dialog-tag/models/distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['dropout_59']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /root/.dialog-tag/models/distilbert-base-uncased and are newly initialized: ['dropout_19']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Datasets Loadings

In [3]:
# loadding the reciepes dataset
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/APUNTS/IntroduccióalesTécniquesdeProcessamentdeLlenguatgeNatural/Project
#%cd /content/drive/MyDrive/NLP

df_recipes = pd.read_csv('ReciepeDataset/RecipeNLG_dataset.csv')
with open('IngredientsDS/baseIngredients.txt') as f:
  lines = f.readlines()

ingredients_name_stopwords = lines[0].split(",")

# reading the dataframe with sentences for applying sentence similarity
df_all = pd.read_excel('sentences_types_dataset.xlsx')

Mounted at /content/drive
/content/drive/MyDrive/APUNTS/IntroduccióalesTécniquesdeProcessamentdeLlenguatgeNatural/Project


In [4]:
# removing the unecessary columns of the dataset
df_recipes = df_recipes.drop(columns=['Unnamed: 0', 'source'])
# displaying the first 10 rows of the dataset
display(df_recipes.head(10))

,title,ingredients,directions,link,NER
0,No-Bake Nut Cookies,"[""1 c. firmly packed brown sugar"", ""1/2 c. eva...","[""In a heavy 2-quart saucepan, mix brown sugar...",www.cookbooks.com/Recipe-Details.aspx?id=44874,"[""brown sugar"", ""milk"", ""vanilla"", ""nuts"", ""bu..."
1,Jewell Ball'S Chicken,"[""1 small jar chipped beef, cut up"", ""4 boned ...","[""Place chipped beef on bottom of baking dish....",www.cookbooks.com/Recipe-Details.aspx?id=699419,"[""beef"", ""chicken breasts"", ""cream of mushroom..."
2,Creamy Corn,"[""2 (16 oz.) pkg. frozen corn"", ""1 (8 oz.) pkg...","[""In a slow cooker, combine all ingredients. C...",www.cookbooks.com/Recipe-Details.aspx?id=10570,"[""frozen corn"", ""cream cheese"", ""butter"", ""gar..."
3,Chicken Funny,"[""1 large whole chicken"", ""2 (10 1/2 oz.) cans...","[""Boil and debone chicken."", ""Put bite size pi...",www.cookbooks.com/Recipe-Details.aspx?id=897570,"[""chicken"", ""chicken gravy"", ""cream of mushroo..."
4,Reeses Cups(Candy),"[""1 c. peanut butter"", ""3/4 c. graham cracker ...","[""Combine first four ingredients and press in ...",www.cookbooks.com/Recipe-Details.aspx?id=659239,"[""peanut butter"", ""graham cracker crumbs"", ""bu..."
5,Cheeseburger Potato Soup,"[""6 baking potatoes"", ""1 lb. of extra lean gro...","[""Wash potatoes; prick several times with a fo...",www.cookbooks.com/Recipe-Details.aspx?id=20115,"[""baking potatoes"", ""extra lean ground beef"", ..."
6,Rhubarb Coffee Cake,"[""1 1/2 c. sugar"", ""1/2 c. butter"", ""1 egg"", ""...","[""Cream sugar and butter."", ""Add egg and beat ...",www.cookbooks.com/Recipe-Details.aspx?id=210288,"[""sugar"", ""butter"", ""egg"", ""buttermilk"", ""flou..."
7,Scalloped Corn,"[""1 can cream-style corn"", ""1 can whole kernel...","[""Mix together both cans of corn, crackers, eg...",www.cookbooks.com/Recipe-Details.aspx?id=876969,"[""cream-style corn"", ""whole kernel corn"", ""cra..."
8,Nolan'S Pepper Steak,"[""1 1/2 lb. round steak (1-inch thick), cut in...","[""Roll steak strips in flour."", ""Brown in skil...",www.cookbooks.com/Recipe-Details.aspx?id=375254,"[""tomatoes"", ""water"", ""onions"", ""Worcestershir..."
9,Millionaire Pie,"[""1 large container Cool Whip"", ""1 large can c...","[""Empty Cool Whip into a bowl."", ""Drain juice ...",www.cookbooks.com/Recipe-Details.aspx?id=794547,"[""pineapple"", ""condensed milk"", ""lemons"", ""pec..."


In [5]:
# we group the sentences needed in each different operation to avoid missundertanding
df_1 = df_all[[9,10]] # ingredient or suggestion
df_2 = df_all[[1,2,3,4,5]] # repeat last step , this step, don't want to continue, want other recipe or keep going
df_3 = df_all[[6,7]] # repeat step or link
df_4 = df_all[[12,13,15]] # new recipe or end conversation
df_5 = df_all[[5,6,13]] # ask for the link or end conversation
df_6 = df_all[[3,8,12,13,14,15]] # when suggesting new recipes, if the user does not like them
df_7 = df_all[[10]]#Verify if the user is asking for a suggestion
df_8 = df_all[[4, 8, 12, 13, 15]] # when showing the possible recipes

## Auxiliar Function and Global Variables Definitions

In [6]:
# auxiliar function to clean the characters of the texts
def clean(text):
  # removing spaces
  text = text.strip()
  # converting to lowercase
  text = text.lower()
  # romanizing
  nfkd_form = unicodedata.normalize('NFKD', text)
  only_ascii  = nfkd_form.encode('ASCII', 'ignore')
  text = only_ascii .decode('utf-8') # bytes to str
  # removing anything that is not a letter or digit
  text = re.sub(r"[^a-zA-Z ]", "", text)
  # double spaces to single
  text = re.sub(' +', ' ', text)
  return text

In [7]:
# auxiliar function to take the ingredients of each row
def clean_ingr_list(ingredients_list):
  final_ingredient_list = []
  ingredients_list = ingredients_list.replace("[","")
  ingredients_list = ingredients_list.replace("]","")
  ingredients_list = ingredients_list.replace("\"","")
  ingredients_list = ingredients_list.split(",")
  for ingredient in ingredients_list: final_ingredient_list.append(ingredient.strip())
  return final_ingredient_list

In [8]:
# Ingredients extraction
# Global variable where all the ingredients (keywords) in a list of list are saved to be able to search throgh them
ingredients = []

for ingredients_list in df_recipes['NER']:
  ingredients_list_cleared = clean_ingr_list(ingredients_list)
  ingredients.append(ingredients_list_cleared)

In [9]:
# Auxiliar function to return the lists of ingredients (keywords) where the input is found
def ingredient_check(input):

    # the input is cleaned and splited in to words
    input = clean(input)
    input  = input.split(" ")

    ingredient_found_list = []

    for word in input:
      for ingredient_list in ingredients :
        #Try spacy
        if word in ingredient_list and (word in ingredients_name_stopwords or lemmatizer.lemmatize(word) in ingredients_name_stopwords):
          ingredient_found_list.append(word)
          break

    return ingredient_found_list

In [10]:
# Auxiliar function to obtain a list of ids where the ingredients
# passed as input appear
def recipes_ids(ingredients_input):
  recipes_ids = []
  id = 0
  for key_ingredients in ingredients:
    # if the input is in the selected list
    if(set(ingredients_input).issubset(set(key_ingredients))):
      recipes_ids.append(id)
    id += 1

  return recipes_ids

In [11]:
# Auxiliar function to get all the information of the recepies where
# the ingredients list input appear
def recipes(ingredients_input):
  # getting the ids with the function recipes_ids()
  ids = recipes_ids(ingredients_input)
  recipes = []
  iterable = 0
  max_recipes = 5
  random.shuffle(ids)
  for id in ids:
    # select the entire row of the dataset
    recipes.append(df_recipes.loc[id])
    if max_recipes <= iterable : break
    else: iterable +=1
  return recipes

In [12]:
# Auxiliar function to obtain 5 different random recipes to suggest to the user
def getRandomRecipes():
  ids = []
  iterable = 0
  for iterable in range(5): ids.append(random.randint(0,len(df_recipes) - 1))
  recipes = []
  iterable = 0
  max_recipes = 5
  for id in ids:
    # select the entire row of the dataset
    recipes.append(df_recipes.loc[id])
    if max_recipes <= iterable : break
    else: iterable +=1
  return recipes

In [13]:
# auxiliar function to get the link of the recipe
def get_link(recipe_id):
  return df_recipes.loc[recipe_id]['link']

In [14]:
# Auxiliar function to dectect if its a yes a no, or something else input (using a dialogue clasifyer)
def response_yes_no(input):
  output = dialog_model.predict_tag(input)
  return output

In [15]:
# Auxiliar function to get the type of the question (using sentence similarity)
def sen_type(df_sentences, input):
  # computing the embedding of the input
  input_embed = model.encode(input, convert_to_tensor=True)

  # Creation of a dataframe with sentence similarity between the input
  # and the sentences in df_sentences dataframe
  column_names = ["Sentence", "Type", "Score"]
  similarities = pd.DataFrame(columns = column_names)

  idx = 0


  for (colname,colval) in df_sentences.iteritems():
    for i in range(len(colval.values)):
        # compute embedding of the sentence in the df
        sen_embed = model.encode(colval.values[i], convert_to_tensor=True)
        # computation of the sentence similarity
        score = float(util.pytorch_cos_sim(input_embed, sen_embed))
        # store the values in similarities dataframe
        similarities.loc[idx] = [colval.values[i], colname, score]
        idx += 1

  # sorting the dataframe by the value 'Score' in ascending order
  similarities = similarities.sort_values(by='Score', ascending=False)
  #print(similarities)
  #print(similarities.drop_duplicates(subset='Type'))
  #print("\n(Maximum correlation score obtained:\n", similarities.iloc[0][2])
  print("\n")
  if similarities.iloc[0][2] < 0.3: return -1
  # return the type of the first row
  else: return similarities.iloc[0][1]

In [16]:
# Auxiliar function to clean the recipe steps in the cell 'directions'
def direction_cleaner(directions):
  directions = directions.replace("]", "")
  directions = directions.replace("[", "")
  directions = directions.replace(",", "")
  directions = directions.replace('"', "")
  directions = directions.split(".")
  # we reomve the last element that is a ' '
  return directions[0:len(directions)-1]

In [17]:
# Auxiliar function to clean the ingredients quantities in the cell 'ingredients'
def ing_quantities(ing_quantites):
  ing_quantites = ing_quantites.replace("]", "")
  ing_quantites = ing_quantites.replace("[", "")
  ing_quantites = ing_quantites.replace('"', "")
  return ing_quantites.split(",")

In [18]:
# global variable of different responses (to avoid repetitions) when the chatbot
# have not understood the user input
chatBotResponses = ['\033[1m' +"Sorry I can not understand you, could you reprhrase your sentence?\n"+'\033[0m',
                    '\033[1m' +"Sorry, could you you rephrase please? I can not understand you.\n"+'\033[0m',
                    '\033[1m' +"Please, input a valid number from the list\n"+'\033[0m']

In [19]:
# Auxiliar function to print the understanding of the chatbot
def understand_print(ptype, incoherence):
  if incoherence > 1:
    print('\033[1m' +"Sorry, it seems that you are looking for other service, I can't understand you.\n\n   ***BYE***"+'\033[0m')
    return True
  else:
    print(chatBotResponses[ptype])
    return False

In [20]:
# global variable of different responses (to avoid repetitions) when
# initiating the conversation
start_questions = ["What do you want to cook? Any preferences or ingredients?",
                   "What do you fancy to prepare? Any specific ingredient?",
                   "With which ingredients do you want to experiment?",
                   "Do you want some meal with particular ingredients?",
                   "Do you want any ideas or do you already know what products are you going to cook?",
                   "Could you tell me what you want to cook or some base ingredients that you want to use?"]

In [22]:
!pip freeze

absl-py==1.4.0
aiohttp==3.8.4
aiosignal==1.3.1
alabaster==0.7.13
albumentations==1.2.1
altair==4.2.2
anyio==3.7.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
array-record==0.4.0
arviz==0.15.1
astropy==5.2.2
astunparse==1.6.3
async-timeout==4.0.2
attrs==23.1.0
audioread==3.0.0
autograd==1.6.1
Babel==2.12.1
backcall==0.2.0
beautifulsoup4==4.11.2
bleach==6.0.0
blis==0.7.9
blosc2==2.0.0
bokeh==2.4.3
branca==0.6.0
build==0.10.0
CacheControl==0.13.1
cached-property==1.5.2
cachetools==5.3.1
catalogue==2.0.8
certifi==2023.5.7
cffi==1.15.1
chardet==4.0.0
charset-normalizer==2.0.12
chex==0.1.7
click==8.1.3
click-plugins==1.1.1
cligj==0.7.2
cloudpickle==2.2.1
cmake==3.25.2
cmdstanpy==1.1.0
colorcet==3.0.1
colorlover==0.3.0
community==1.0.0b1
confection==0.0.4
cons==0.4.6
contextlib2==0.6.0.post1
contourpy==1.1.0
convertdate==2.4.0
cufflinks==0.17.3
cvxopt==1.3.1
cvxpy==1.3.1
cycler==0.11.0
cymem==2.0.7
Cython==0.29.35
dask==2022.12.1
datascience==0.17.6
db-dtypes==1.1.1
dbus-p

## Chatbot Implementation

In [21]:
#MAIN COMPONENT
# var to know if the conversation just begun
start = 0
# var to know in which mode the conversation is
mode = 0
# var to keep count of incoherence of the user
incoherence = 0
# var to be able to keep a track of the recipe step
recipe_step_counter = 0
# var to know if the first step is given
first = 0
#var to to be able to show other reciepes to the user and keep track of the current recipe
recipe_idx_count = 0
#list of possible recipes to provide
possible_recipes = []
userInput = ""
# bool var to know if we are in suggesting mode
suggest = False
while(True):
  # MODE 0 --> Starting the conversation
  if mode == 0:
    #print("\n(mode: %d)\n" %mode)
    recipe_idx_count = 0
    # to print this sentence just at the beginning
    if start == 0:
      print('\033[1m' +"Hello! I am your culinary assistant.")
      start = 1
    print(random.choice(start_questions), '\n')
    print('\033[0m')
    userInput = input('Write your response:\n')
    print("\n")
    # first of all, we check if there are any specific ingredients on the input
    # and store it
    ing = ingredient_check(userInput)

    # if there are ingredients in the input
    if (len(ing) > 0):
      #print("\n(Entities extracted:", ing, ')\n')
      # search for recipes with those specific ingredients
      possible_recipes = recipes(ing)
      if (len(possible_recipes) > 0):
        suggest = False
        recipe_title = possible_recipes[recipe_idx_count][0]
        print('\033[1m' +"Are you interested in the recepie %sor do you want to know more about it? (Answer with Yes or No)\n" %recipe_title +'\033[0m')
        mode = 1

      # if there is no recipes with those ingredients
      else:
        print('\033[1m' +"I haven't found any reciepe with this ingredients. Please, change ingredient names or if you need a suggestion, let me know.\n"+'\033[0m')
        userInput =  input('Write your response:\n')
        mode = 0

    # if there is no ingredients in the input
    else:
      result = int(sen_type(df_7, userInput))
      #print("(Predicted class: %d)\n" %result)
      if result == 10:
        print("\033[1mFine, I can recommend you some nice recipes:\n \033[0m")
        mode = 4
      else:
        if(understand_print(0,incoherence)): break


  # MODE 1 --> classifying y/n answer
  if mode == 1:
    #print("\n(mode: %d)\n" %mode)
    userInput =  input('Write your response:\n')
    response = response_yes_no(userInput)
    #print("(Predicted class: %s)\n" %response)
    if response == "Acknowledge (Backchannel)" or  response == "Agree/Accept" or response =="Appreciation":
      incoherence = 0
      mode = 2
    elif response == "No Answers":
      incoherence = 0
      mode = 6
    else:
      result = int(sen_type(df_8, userInput))
      #print("(Predicted class: %d)\n" %result)
      if result == 4 or result == 8 or result == 13: mode = 8
      if result == 12 or result == 15: mode = 0
      if result == -1:
        if(understand_print(0, incoherence)):  break

  # MODE 2 --> give ingredients of the recipe
  if mode == 2:
    #print("\n(mode: %d)\n" %mode)
    ingrednients_show = possible_recipes[recipe_idx_count][4]
    print("\033[1m Okey, these are the recipe ingredients %s, shall we continue?(Answer with Yes or No)\n \033[0m" %(ingrednients_show))
    userInput =  input('Write your response:\n')
    print("\n")

    response = response_yes_no(userInput)
    #print("(Predicted class: %s)\n" %response)
    if response == "Acknowledge (Backchannel)" or  response == "Agree/Accept" or response == "Appreciation":
      incoherence = 0
      recipe_step_counter = 0
      print("\033[1m These are the ingredients and their quantities in the %s recipe: \033[0m" %recipe_title)
      # listing the ingredients needed with its exactly quantities
      for product in ing_quantities(df_recipes['ingredients'][recipe_idx_count]):
        print("- ", product.strip())
      mode = 3

    elif response == "No Answers":
      incoherence = 0
      print("\033[1m Fine, do you want me to show you more recipes or do you want to try with new ingredients? \033[0m")
      userInput =  input('Write your response:\n')
      print("\n")
      result = int(sen_type(df_6, userInput))
      #print("(Predicted class: %d)\n" %result)
      if result == 3 or result == 12 or result == 15: mode = 0
      if result == 14: mode = 2
      if result == 13 or result == 8: mode = 8

    else:
      result = int(sen_type(df_8, userInput))
      #print("(Predicted class: %d)\n" %result)
      if result == 4 or result == 8 or result == 13: mode = 8
      if result == 12 or result == 15: mode = 0
      if result == -1:
        if(understand_print(0, incoherence)):  break


  # MODE 3 --> give steps of the recipe
  if mode == 3:
    #print("\n(mode: %d)\n" %mode)
    # if the orders just started
    if (recipe_step_counter == 0 and first == 0):
      first = 1
      print("\033[1mPerfect, let's start\nIf you get lost or you need help, please let me now\n \033[0m")
      # retaining and cleaning the directions
      directions = direction_cleaner(possible_recipes[recipe_idx_count][2])

    # if the orders haven't finished
    if(recipe_step_counter < len(directions)-1):
      print("\033[1mStep %s : %s\n \033[0m" %(recipe_step_counter, directions[recipe_step_counter]))
      recipe_step_counter += 1
      print("\033[1m(If you want to continue enter 1)\n \033[0m")
      userInput =  input('Write your response:\n')
      print("\n")
      if userInput == "1":
        mode == 3
      else:
        mode = 9

    # the user has finished the recipe
    if (recipe_step_counter == len(directions)-1):
      print("\033[1m The recipe is finished. I hope you have liked it!\n \033[0m")
      mode = 8

  # MODE 4 --> suggest recipes
  if mode == 4:
    #print("\n(mode: %d)\n" %mode)
    recipe_idx_count = 0
    possible_recipes = getRandomRecipes()
    suggest = True
    recipe_title = possible_recipes[recipe_idx_count][0]

    print("\033[1m Are you interested in the recepie %s? (Answer with Yes or No)\n \033[0m" %recipe_title)
    mode = 1

  # MODE 5 --> provide the link
  if mode == 5:
    #print("\n(mode: %d)\n" %mode)
    print("\033[1m Here you have the original source of the recipe %s: %s \n \033[0m" %(recipe_title, get_link(recipe_idx_count)))
    if(recipe_step_counter < len(directions)-1): mode = 3
    else: mode = 8

  # MODE 6 --> change recipe
  if mode == 6:
    #print("\n(mode: %d)\n" %mode)
    recipe_idx_count +=1
    if len(possible_recipes) > recipe_idx_count:
      recipe_title = possible_recipes[recipe_idx_count][0]
      print("\033[1m Are you interested in the recipe %s? (Answer with Yes or No)\n \033[0m" %recipe_title)
      mode = 1
    else:
      if suggest == True:
        print("\033[1m Remember that you can ask me for certain ingredients.\n \033[0m")
        mode = 4
      else:
        print(" \033[1m I don't have any more recipes with that characteristics (ingredients), could you choose any other ingredients?\n \033[0m")
        mode = 0
        userInput =  input('Write your response:\n')
        print("\n")

  # MODE 7 --> the user got lost during the process
  if mode == 7:
    #print("\n(mode: %d)\n" %mode)
    print("\033[1m Please let me know in which step did you got lost. [enter the exact number of step]\n \033[0m")
    steps = np.arange(recipe_step_counter)
    print(np.arange(recipe_step_counter))
    userInput =  input('Write your response:\n')
    print("\n")
    if (userInput.isnumeric()):
      userInput = int(userInput)
      if userInput in steps:
        incoherence = 0
        recipe_step_counter = userInput
        mode = 3
      else:
        understand_print(2, incoherence)
    else:
      understand_print(2, incoherence)

# MODE 8 --> the recipe is finished, the user is asked to leave the conversation or start again
  if mode == 8:
    #print("\n(mode: %d)\n" %mode)
    print("\033[1m Do you want to start another recipe or just finish? \033[0m")
    userInput = input('Write your response:\n')
    print("\n")
    result = int(sen_type(df_4, userInput))
    #print("(Predicted class: %d)\n" %result)
    if result == 12 or result == 15: mode = 0
    if result == 13: mode = 11


  # MODE 9 --> resolving problems when showing the recipe
  if mode == 9:
    #print("\n(mode: %d)\n" %mode)
    #apply sentence similarity
    result = int(sen_type(df_2, userInput))
    #print("(Predicted class: %d)\n" %result)
    if result == 1: mode = 7
    if result == 2: mode = 10
    if result == 3:
      print("\033[1m Okey, let's start again\n \033[0m")
      mode = 0
    if result == 4:
      print("\033[1m Okey, so do you want to end the conversation or start a new recipe?\n \033[0m")
      userInput = input('Write your response:\n')
      print("\n")
      result = int(sen_type(df_4, userInput))
      if result == 12: mode = 0
      else:
        print('\033[1m' + "***BYE***"+'\033[0m')
        break
    if result == 5: mode = 3
    if result == -1:
      understand_print(0, incoherence)


  # MODE 10 --> repeat the step or provide the link
  if mode == 10:
   # print("\n(mode: %d)\n" %mode)
    print("\033[1m Do you want me to repeat it? Or I can provide you with a link if its more clear for you\n \033[0m")
    userInput = input('Write your response:\n')
    print("\n")
    result = int(sen_type(df_3, userInput))
    #print("(Predicted class: %d)\n" %result)
    if result == 7:
      recipe_step_counter -= 1
      mode = 3
    else:
      mode = 5

  # MODE 11 --> the recipe has finished, asking for the original source
  if mode == 11:
    #print("\n(mode: %d)\n" %mode)
    print("\033[1m Before finishing, do you want to keep the recipe?\n \033[0m")
    userInput = input('Write your response:\n')
    print("\n")
    result = int(sen_type(df_5, userInput))
    #print("(Predicted class: %d)\n" %result)
    if result == 6 or result == 5:
      #print("result: %d\n" %result)
      print('\033[1m Here you have the original source of the recipe %s: %s \n\033[1m' %(recipe_title, get_link(recipe_idx_count)))
    break



Hello! I am your culinary assistant.
Do you want some meal with particular ingredients? 




KeyboardInterrupt: ignored